In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
from visualization import TrainingVisualizer

In [2]:
# visualization.py
# import matplotlib.pyplot as plt
# from matplotlib.ticker import MaxNLocator
# from IPython import display

# class TrainingVisualizer:
#     """在动画中绘制数据并记录最高的acc和最低的loss"""
#     def __init__(self, xlabel=None, ylabel=None, title=None, legend=None, xlim=None,
#                  ylim=None, xscale='linear', yscale='linear',
#                  fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
#                  figsize=(7, 5)):
#         if legend is None:
#             legend = []
#         self.fig, self.axes = plt.subplots(nrows, ncols, figsize=figsize)
#         if nrows * ncols == 1:
#             self.axes = [self.axes, ]
#         self.config_axes = lambda: self.set_axes(
#             self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
#         self.X, self.Y, self.fmts = None, None, fmts
#         self.legend = legend
#         self.title = title  # 保存标题

#         # 新增：记录最高acc和最低loss
#         self.best_acc = float('-inf')  # 初始化为负无穷
#         self.min_loss = float('inf')   # 初始化为正无穷

#     def set_axes(self, ax, xlabel, ylabel, xlim, ylim, xscale, yscale, legend):
#         """设置matplotlib的轴"""
#         ax.set_xlabel(xlabel)
#         ax.set_ylabel(ylabel)
#         ax.set_xscale(xscale)
#         ax.set_yscale(yscale)
#         if xlim:
#             ax.set_xlim(xlim)
#         if ylim:
#             ax.set_ylim(ylim)
#         if legend:
#             ax.legend(legend)
#         ax.xaxis.set_major_locator(MaxNLocator(integer=True))  # 设置x轴刻度为整数
#         ax.grid()
        
#     def add(self, x, y):
#         if not hasattr(y, "__len__"):
#             y = [y]
#         n = len(y)
#         if not hasattr(x, "__len__"):
#             x = [x] * n
#         if self.X is None:
#             self.X = [[] for _ in range(n)]
#         if self.Y is None:
#             self.Y = [[] for _ in range(n)]
#         for i, (a, b) in enumerate(zip(x, y)):
#             if a is not None and b is not None:
#                 self.X[i].append(a)
#                 self.Y[i].append(b)

#                 # 更新最高acc和最低loss
#                 if self.legend[i].lower().find('acc') != -1:  # 检查是否是acc数据
#                     if b > self.best_acc:
#                         self.best_acc = b
#                 if self.legend[i].lower().find('loss') != -1:  # 检查是否是loss数据
#                     if b < self.min_loss:
#                         self.min_loss = b

#         self.axes[0].cla()
#         for x, y, fmt in zip(self.X, self.Y, self.fmts):
#             self.axes[0].plot(x, y, fmt)
#         self.config_axes()

#         # 显示最高acc和最低loss
#         if self.title:
#             self.axes[0].set_title(self.title + f"\nBest Acc: {self.best_acc:.4f}, Min Loss: {self.min_loss:.4f}")
#         else:
#             self.axes[0].set_title(f"Best Acc: {self.best_acc:.4f}, Min Loss: {self.min_loss:.4f}")
        
#         display.display(self.fig)
#         display.clear_output(wait=True)

# # 调用示例
# # visualizer = TrainingVisualizer(xlabel='Epoch', ylabel='Value', title='Train TextRCNN', legend=['training_loss', "training_acc", "testing_loss", "testing_acc"]) # 初始化
# # for epoch in range(num_epochs):
# #     train_acc, train_loss = 
# #     val_acc, val_loss = 
# #     visualizer.add(epoch, [train_loss, train_acc, val_loss, val_acc])


In [3]:
# 检查GPU是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# 加载文本数据
with open('data/time_machine_txt/timemachine.txt', 'r') as f:
    text = f.read()

# 创建字符映射表
chars = sorted(list(set(text)))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
idx_to_char = {idx: ch for idx, ch in enumerate(chars)}

# 转换文本为索引
text_as_int = np.array([char_to_idx[c] for c in text])

In [5]:
# 定义超参数
seq_length = 100  # 序列长度 - 一个句子100个单词
batch_size = 256
hidden_size = 256
embedding_dim = 64  # input_size 嵌入向量的大小
num_layers = 1
learning_rate = 0.001
num_epochs = 10
vocab_size = len(chars)  # 字符的个数

def create_dataset(text_as_int, seq_length, batch_size):
    sequences = []
    targets = []
    for i in range(0, len(text_as_int) - seq_length):
        sequences.append(text_as_int[i:i + seq_length])
        targets.append(text_as_int[i + 1:i + seq_length + 1])
    sequences = torch.tensor(sequences, dtype=torch.long)
    targets = torch.tensor(targets, dtype=torch.long)
    dataset = torch.utils.data.TensorDataset(sequences, targets)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

dataloader = create_dataset(text_as_int, seq_length, batch_size)

C:\Users\20703\AppData\Local\Temp\ipykernel_12996\3710666413.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  sequences = torch.tensor(sequences, dtype=torch.long)


In [6]:
# 定义GRU模型
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        out, hidden = self.gru(x, hidden)
        out = out.contiguous().view(-1, hidden.size(2))
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.randn(num_layers, batch_size, hidden_size).to(device)

In [ ]:
num_epochs = 10

# 实例化模型
model = GRUModel(vocab_size, embedding_dim, hidden_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
visualizer = TrainingVisualizer(xlabel='Epoch', ylabel='Value', title='Train GRU in Time_Machine', legend=['Train Loss', 'Perplexity'])

# 训练模型并计算困惑度
for epoch in range(num_epochs):
    total_loss = 0
    for i, (inputs, targets) in enumerate(dataloader):
        current_batch_size = inputs.size(0)
        inputs = inputs.view(current_batch_size, -1).to(device)
        targets = targets.view(-1).to(device)

        optimizer.zero_grad()
        hidden = model.init_hidden(current_batch_size).detach()
        output, hidden = model(inputs, hidden)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    perplexity = torch.exp(torch.tensor(avg_loss))
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Perplexity: {perplexity:.4f}')

    visualizer.add(epoch, [avg_loss, perplexity.item()])


KeyboardInterrupt



Error in callback <function _draw_all_if_interactive at 0x0000027970C4EB80> (for post_execute):


In [ ]:
# 生成文本
def generate_text(model, start_str, length):
    model.eval()
    hidden = model.init_hidden(1)
    input = torch.tensor([char_to_idx[ch] for ch in start_str], dtype=torch.long).unsqueeze(0).to(device)

    generated_text = start_str
    for _ in range(length):
        output, hidden = model(input, hidden)
        _, top_idx = torch.topk(output[-1], 1)
        next_char = idx_to_char[top_idx.item()]
        generated_text += next_char
        input = torch.tensor([[top_idx]], dtype=torch.long).to(device)
    
    return generated_text

# 生成文本示例
start_str = "Filby became pensive. "
generated_text = generate_text(model, start_str, 100)
print(generated_text)